In [ ]:
#myRobot.py

from math import pi, sin, cos, sqrt, exp
from numpy import array, dot

from Robot3WD.Robot3WD import Robot

class myRobot(Robot):
    def __init__(self, sampling_period):
        Robot.__init__(self, sampling_period)

    def inverse_kinematics(self, p_dot, theta):
        L = self._L
        wheel_radius = self._wheel_radius
        M = array([[sin(theta), (-1)*cos(theta), (-1)*L], [cos((pi/6)+theta), sin((pi/6)+theta), (-1)*L], [(-1)*cos((pi/6)-theta), sin((pi/6)-theta), (-1)*L]])
        wheel_angular_velocities = (1/wheel_radius)*(dot(M, p_dot))
        return wheel_angular_velocities

    def forward_kinematics(self, wheel_angular_velocities, theta):
        L = self._L
        wheel_radius = self._wheel_radius
        M = array([[2*sin(theta), 2*cos(theta+(pi/6)), (-2)*sin(theta+(pi/3))], [-2*cos(theta), 2*cos(theta-(pi/3)), 2*cos(theta+(pi/3))], [(-1)/L, (-1)/L, (-1)/L]])
        p_dot = (wheel_radius/3)*(dot(M, wheel_angular_velocities))
        return p_dot

    def motor_limit(self, wheel_angular_velocities, limit_value):
    # Initialize the output list
      wheel_angular_velocities_bar = [0] * len(wheel_angular_velocities)

      for i in range(len(wheel_angular_velocities)):
          if wheel_angular_velocities[i] < -5 * limit_value:
              wheel_angular_velocities_bar[i] = -5 * limit_value
          elif -5 * limit_value <= wheel_angular_velocities[i] <= 5 * limit_value:
              wheel_angular_velocities_bar[i] = wheel_angular_velocities[i]
          elif wheel_angular_velocities[i] > 5 * limit_value:
              wheel_angular_velocities_bar[i] = 5 * limit_value

      return wheel_angular_velocities_bar

    def move_left(self, v, theta):
        p_dot = [-v, 0.0, 0.0]
        w = self.inverse_kinematics(p_dot, theta)
        self.set_angular_velocities(w)

    def move_forward(self, v, theta):
        p_dot = [0.0, v, 0.0]
        w = self.inverse_kinematics(p_dot, theta)
        self.set_angular_velocities(w)

    def move_backward(self, v, theta):
        p_dot = [0.0, -v, 0.0]
        w = self.inverse_kinematics(p_dot, theta)
        self.set_angular_velocities(w)

    def move_right(self, v, theta):
        p_dot = [v, 0.0, 0.0]
        w = self.inverse_kinematics(p_dot, theta)
        self.set_angular_velocities(w)

    def rotate_CCW(self, w, theta):
        p_dot = [0.0, 0.0, w] #<- positive since when testing on bot, with w set to negative the robot would rotate CW instead of CCW
        w = self.inverse_kinematics(p_dot, theta)
        self.set_angular_velocities(w)

    def rotate_CW(self, w, theta):
        p_dot = [0.0, 0.0, -w] #<- negative since when testing on bot, with w set to positive the robot would rotate CCW instead of CW
        w = self.inverse_kinematics(p_dot, theta)
        self.set_angular_velocities(w)

    def HMatrix(q):
        H = array([[cos(q[2]), -sin(q[2]), q[0]], [sin(q[2]), cos(q[2]), q[1]], [0, 0, 1]])
        return H

    def Vraw_to_distance(Vraw):
        d = 0.6209*exp(-0.05396*sqrt(Vraw))
        return d

In [ ]:
#lab4demo.py

from myRobot import myRobot
from Robot3WD.PID import PIDControllers
from numpy import array, dot, subtract, zeros
from numpy.linalg import norm
from math import pi, sqrt, sin, cos, atan2
import os
import time
import numpy as np

# Initialize sampling period & stopping time
T_s = 0.02
T_f = 20.0
robot = myRobot(T_s)

# Initialize initial posture (x_0, y_0, theta_0)
x_0 = 0.0
y_0 = 0.0
theta_0 = pi / 6

# Initialize final posture (x_f, y_f, theta_f)
x_f = 0.7
y_f = 1.0
theta_f = pi / 2

# Initialize p_0, p_f
p_0 = array([x_0, y_0, theta_0]).T
p_f = array([x_f, y_f, theta_f]).T

# Initialize d_min and d_free
d_min = 0.08
d_free = 1.25*d_min

# Initialize stopping criterion (epsilon)
epsilon = 0.01

# Define the controller gains k_rho, k_beta and k_alpha
k_rho = 1.0
k_beta = -1.0
k_alpha = (2.0/pi)*k_rho - (5.0/3.0)*k_beta + 0.5

# Create an instance of the PID controllers object and initialize it
pid = PIDControllers(Kp, Ki, Kd, T_s)

# Initialize robot & other variables (goal_reached, current_time, p)
robot.initialize(theta_0)
goal_reached = False
current_time = 0.0
p = p_0

# Open a file to store data for plotting
with open('p.csv', 'w') as f:  # Use 'with' to ensure the file is properly closed
    # Set start time to current time
    start_time = time.time()

    # Control loop
    while not goal_reached and current_time < T_f:
        # Get current robot orientation (theta)
        robot.get_readings_update()
        theta = robot.orientation

        # Determine pd_dot (desired velocities) using PID controller
        pd_dot = pid(p_f, p)

        # Determine desired wheel velocities using inverse kinematics
        wheel_angular_velocities = robot.inverse_kinematics(pd_dot, theta)

        # Execute the motion by sending desired angular velocities to the robot's motors
        robot.set_angular_velocities(wheel_angular_velocities)

        # Update odometry
        p += pd_dot * T_s  # Update position based on desired velocities

        # Replace angle with the measured value from IMU
        p[2] = robot.orientation

        # Check to see if the goal is reached
        error_norm = np.linalg.norm(subtract(p_f, p))
        if error_norm < epsilon:
            goal_reached = True

        # Time update
        current_time = time.time() - start_time

        # Write data to file (current time, p[0], p[1], p[2], error_norm)
        f.write('%10.3e %10.3e %10.3e %10.3e %10.3e\n' % (current_time, p[0], p[1], p[2], error_norm))

# Print the elapsed time
print("Elapsed time is", current_time)

# Either goal is reached or current_time > T_f
if goal_reached:
    print("Goal is reached, error norm is:", error_norm)
else:
    print("Goal is not reached, error norm is:", error_norm)

In [ ]:
#lab4demo.py (from Dexter)

from time import time
from math import sqrt, pi, sin, cos, atan2
from numpy import array, dot, zeros
from numpy.linalg import norm

from myRobot import *
import sys
# Set up sampling period T_s and stopping time T_f
T_s = 0.02
T_f = 30

# Set up initial pose
x_0 = 0
y_0 = 0
theta_0 = 0

# Set up goal position
x_f = 1.3
y_f = 1.5

# Set up p_0 and p_f
p_0 = array([x_0, y_0, theta_0])
p_f = array([x_f, y_f, 0])

# Set up error tolerance
epsilon = sqrt( 2.0*(0.5/100.0)**2)

# set up d_free and d_min
d_min = 0.08
d_free = 1.25*d_min

# Set up controller gains
k_rho = 1.0
k_beta = -1.0
k_alpha = (2.0/pi)*k_rho - (5.0/3.0)*k_beta + 0.5

# Initialize vector pr_dot to be used for inverse kinematics
pr_dot = zeros(shape=(3,1))

# Initialize vector d for storing the sensor measurements d_i
d = array([10.0, 10.0, 10.0, 10.0, 10.0, 10.0]).T

# Initialize u_iR
u_iR = zeros(shape=(3,6))

# Initialize sensor frame location
R = 0.13
halfR = R/2.0
root3Rover2 = sqrt(3.0)*R/2.0

# Sensor frame location as per Table 4.1
sensor_loc = array([
   [-halfR, -root3Rover2, -(2.0/3.0)*pi],
   [-root3Rover2, halfR, (5/6)*pi],
   [0, R, 1/2*pi],
   [halfR, root3Rover2, 1/3*pi],
   [root3Rover2, halfR, 1/6*pi],
   [root3Rover2, -halfR, -1/6*pi]
])

# open a file to store robot pose for plotting later
f = open('pose.csv', 'w')

# Initial robot and other variables
robot = myRobot(T_s)

robot.initialize(theta_0)

H_R_Si_list = []
for i in range(0,6):
  q = sensor_loc[i]
  H_R_Si = robot.HMatrix(q)
  H_R_Si_list.append(H_R_Si)

H_R_Si_array = array(H_R_Si_list)

p = p_0
dp = p_f - p_0
rho = norm(dp[0:2])
goal_reached = False
elapsed_time = 0.0

# Set start time
start_time = time()


# Control loop
while ( (not goal_reached) and (elapsed_time < T_f) ):

    robot.get_readings_update()

    # save data to file for plotting
    f.write('%10.3e %10.3e % 10.3e %10.3e %10.3e\n' % (elapsed_time, p[0], p[1], p[2], rho))

    x = p[0]
    y = p[1]
    theta = p[2]

    # Use forward kinematics to get p_dot
    p_dot = robot.forward_kinematics(robot.angular_velocities, theta)

    # Determine angle alpha (remember to use atan2 instead of atan)
    alpha = atan2(y_f-y, x_f-x)-theta

    d_si_list = []
    d_accepted = []
    u_R_list = []
    d_R_list = []
    accepted_indices = []

    collision_exist = False

    # Get sensor measurements w.r.t. sensor frames
    for i in range(0,6):
       d_measured = robot.Vraw_to_distance(robot.ir_sensors_raw_values[i])
       d[i] = d_measured

       if d_measured <= d_min:
           collision_exist = True

    if collision_exist:
        for i in range(0,6):
           d_measured = robot.Vraw_to_distance(robot.ir_sensors_raw_values[i])

           # find collision free paths

           if d_measured > d_free:
               d_accepted.append(d_measured)
               accepted_indices.append(i)

           # convert this path to vector d_Si
           d_si= array([d_measured, 0, 1])

           # transform from Si to R frame
           d_R_i = dot(H_R_Si_array[i], d_si)
           d_R_list.append(d_R_i)

        # calculated unit path vectors
        u_R_list = [d_R_i/norm(d_R_list) for d_R_i in d_R_list]
        d_T = sum(d_accepted)

        # calculate weighted sum of path vectors
        u_R = sum([d[i]/d_T*u_R_list[i] for i in accepted_indices])

        # transform R to base link frame
        H_0_R = robot.HMatrix(p)
        u_0 = dot(H_0_R, u_R)

        # set temporary goal
        p_T = d_free* u_0

        alpha_bar = atan2(p_T[1]- y, p_T[0]-x) - theta
        alpha = alpha_bar

        rho_bar = norm([p_T[1]- y, p_T[0]-x])
        rho = rho_bar

    beta = -(theta + alpha)

    # Determine kinear and angular velocities of the robot body (v, w)
    # using the control law given in Eqs. (4.12) and (4.13)
    v = k_rho * rho
    w = k_alpha * alpha + k_beta * beta

    # Determine pr_dot (temporary velocity desired)
    pr_dot[0] = v * cos(theta)
    pr_dot[1] = v * sin(theta)
    pr_dot[2] = w

    # Now use Inverse Kinematics to determine wheel ref velocities
    wheel_ref_vel = robot.inverse_kinematics(pr_dot, theta)

    # Apply motor limits
    wheel_ref_vel = robot.motor_sat(wheel_ref_vel, 5.0*pi)

    # Execute motion
    robot.set_angular_velocities(wheel_ref_vel)

    # Odometry update
    p = p + p_dot*T_s

    # Replace calculated update for theta with measured value from IMU
    p[2]= robot.orientation

    # Check to see if goal is reached
    dp = p_f - p
    rho = norm(dp[0:2])
    goal_reached = ( rho <= epsilon)

    # time update
    elapsed_time = time() - start_time


print('ELPASED TIME = %s rho = %s' % (elapsed_time, rho))

# Either goal is reached or current_time > T_f
if goal_reached:
   print('Goal is reached, error norm is', rho)
else:
   print('Failed to reach goal, error norm is', rho)

robot.stop()
robot.close()

f.close()

In [ ]:
#attempt

import os
from numpy import array, dot, zeros
from numpy.linalg import norm
from math import sqrt, pi, sin, cos, atan2
from time import time

from myRobot import *

# Set up sampling period T_s and stopping time T_f
T_s = 0.02      # Sampling Period is 0.02 seconds
T_f = 8.0       # Run Time is 8 seconds

# Set up initial posture
x_0 = 0.0
y_0 = 0.0
theta_0 = 0.0

# Set up goal position
x_f = 0.7
y_f = 1.0
theta_f = 0.0

# Set up p_0 and p_f
p_0 = array([x_0, y_0, theta_0]).T
p_f = array([x_f, y_f, theta_f]).T

# Initialize d_min and d_free (See Eq. 4.4 and 4.5)
d_min = 0.08
d_free = 1.25*d_min

# Set up error tolerance to be within 1cm of target in both x and y directions
epsilon = sqrt( 2.0*(1.0/100.0)**2 )

# Set up controller gains (required in Eq. 4.11 and 4.12)
k_rho = 1.0
k_beta = -1.0
k_alpha = (2.0/pi)*k_rho - (5.0/3.0)*k_beta + 0.5

# Initialize vector pr_dot to be used for inverse kinematics
pr_dot = array([0.0, 0.0, 0.0]).T

# Initialize vector d. The sensor measurement d_i will be assigned to d[i]
d = array([10.0, 10.0, 10.0, 10.0, 10.0, 10.0]).T

# Initialize a 3x6 matrix u_iR. The vector delta_i^R will be assigned to
# the i-th column of this matrix
u_iR = zeros(shape=(3,6))

# Initialize a 6x3 matrix sensor_loc to store the sensor frame
# locations as per Table 4.1. The i-th row of this matrix corresponds
# to the i-th ROW of Table 4.1.
R = 0.13
halfR = R/2.0
root3Rover2 = sqrt(3)*R/2.0
sensor_loc = array([
   [-halfR, -root3Rover2, -(2.0/3.0)*pi],
   [-root3Rover2, halfR, (5.0/6.0)*pi],
   [0, R, (1.0/2.0)*pi],
   [halfR, root3Rover2, (1.0/3.0)*pi],
   [root3Rover2, halfR, (1.0/6.0)*pi],
   [root3Rover2, -halfR, -(1.0/6.0)*pi]
])

# open a file to store data for plotting later
f = open('p.csv', 'w')
os.chmod('p.csv', 0666)

# Initial robot
robot = myRobot(T_s)

robot.initialize(theta_0)

# Compute distance to goal, rho, using Eq. 4.1
dp = p_f - p_0
rho = norm(dp[0:2])  # Note: In Python, the range 0:2 means [0,1] NOT [0,1,2]

# Initialize additional variables
p = p_0
goal_reached = False
elapsed_time = 0.0

# Set start time
start_time = time()

# Control loop
while ( (not goal_reached) and (elapsed_time < T_f) ):

    robot.get_readings_update()

    # save data to file for plotting
    f.write('%10.3e %10.3e % 10.3e %10.3e %10.3e\n' % (elapsed_time, p[0], p[1], p[2], rho))

    theta = p[2]

    # Use forward kinematics to get p_dot
    p_dot = robot.forward_kinematics(robot.angular_velocities, theta)

    # Determine angle alpha (remember to used atan2 instead of atan)
    alpha = atan2(dp[1], dp[0]) - theta

    # Get sensor measurements w.r.t. sensor frames
    for i in range(0, 6):
        d[i] = Vraw_to_distance(robot.ir_sensors_raw_values[i])

    # Check if there is a risk of collision with obstacle(s).
    # If true, determine the temporary goal vectors delta_goal^0 and delta_goal^R
    # and use them to find the temporary alpha and rho.
    # See Eqs. 4.4 to 4.11.
    if min(d) <= d_min:
        x_bar = 0.0
        y_bar = 0.0
        u_iR[0, :] = d
        u_iR[2, :] = 1 # The delta_i^Si matrix (creating the new path avoiding the obstacle)
        for i in range(0, 6):
            if d[i] >= d_free:
                u_iR[:, i] = dot(HMatrix((sensor_loc[i, :]).T), u_iR[:, i])
                x_bar = x_bar + u_iR[0, i]
                y_bar = y_bar + u_iR[1, i]

        delta_goalR = array([x_bar, y_bar, 1]).T    # delta_goal^R = dot(H_Si^R, delta_i^Si)
        alpha = atan2(delta_goalR[i, 1], delta_goalR[i, 0]) - theta

        delta_goal0 = dot(HMatrix(p), delta_goalR)  # delta_goal^0 = dot(H_R^0, delta_goal^R)
        dp = delta_goalR - p_0
        rho = norm(dp[0:2])

    # Determine the angle beta
    beta = -(alpha + theta)

    # Determine linear and angular velocities of the robot (v and w) using the control law
    # given in Eqs. 4.12 and 4.13
    v = k_rho*rho
    w = k_alpha*alpha + k_beta*beta

    pr_dot[0] = v*cos(theta)
    pr_dot[1] = v*sin(theta)
    pr_dot[2] = w

    # Now use Inverse Kinematics to determine wheel ref velocities
    wheel_ref_vel = robot.inverse_kinematics(pr_dot, theta)

    # Execute motion
    robot.set_angular_velocities(wheel_ref_vel)

    # Odometry update
    p = p + (p_dot*T_s)

    # Replace calculated update for theta with measured value from IMU
    p[2]= robot.orientation

    # Check to see if goal is reached
    dp = p_f - p
    rho = norm(dp[0:2])
    goal_reached = ( rho <= epsilon)

    # time update
    elapsed_time = time() - start_time


print 'ELPASED TIME = %s rho = %s' % (elapsed_time, rho)

# Either goal is reached or current_time > T_f
if goal_reached:
   print 'Goal is reached, error norm is', rho
else:
   print 'Failed to reach goal, error norm is', rho

robot.stop()
robot.close()

f.close()

In [ ]:
#myRobot.py (WORKING)

from math import pi, sin, cos, sqrt, exp
from numpy import array, dot

from Robot3WD.Robot3WD import Robot

class myRobot(Robot):
    def __init__(self, sampling_period):
        Robot.__init__(self, sampling_period)

    def inverse_kinematics(self, p_dot, theta):
        L = self._L
        wheel_radius = self._wheel_radius
        M = array([[sin(theta), (-1)*cos(theta), (-1)*L], [cos((pi/6)+theta), sin((pi/6)+theta), (-1)*L], [(-1)*cos((pi/6)-theta), sin((pi/6)-theta), (-1)*L]])
        wheel_angular_velocities = (1/wheel_radius)*(dot(M, p_dot))
        return wheel_angular_velocities

    def forward_kinematics(self, wheel_angular_velocities, theta):
        L = self._L
        wheel_radius = self._wheel_radius
        M = array([[2*sin(theta), 2*cos(theta+(pi/6)), (-2)*sin(theta+(pi/3))], [-2*cos(theta), 2*cos(theta-(pi/3)), 2*cos(theta+(pi/3))], [(-1)/L, (-1)/L, (-1)/L]])
        p_dot = (wheel_radius/3)*(dot(M, wheel_angular_velocities))
        return p_dot

    def motor_limit(self, wheel_angular_velocities, limit_value):
    # Initialize the output list
      wheel_angular_velocities_bar = [0] * len(wheel_angular_velocities)

      for i in range(len(wheel_angular_velocities)):
          if wheel_angular_velocities[i] < -5 * limit_value:
              wheel_angular_velocities_bar[i] = -5 * limit_value
          elif -5 * limit_value <= wheel_angular_velocities[i] <= 5 * limit_value:
              wheel_angular_velocities_bar[i] = wheel_angular_velocities[i]
          elif wheel_angular_velocities[i] > 5 * limit_value:
              wheel_angular_velocities_bar[i] = 5 * limit_value

      return wheel_angular_velocities_bar

    def move_left(self, v, theta):
        p_dot = [-v, 0.0, 0.0]
        w = self.inverse_kinematics(p_dot, theta)
        self.set_angular_velocities(w)

    def move_forward(self, v, theta):
        p_dot = [0.0, v, 0.0]
        w = self.inverse_kinematics(p_dot, theta)
        self.set_angular_velocities(w)

    def move_backward(self, v, theta):
        p_dot = [0.0, -v, 0.0]
        w = self.inverse_kinematics(p_dot, theta)
        self.set_angular_velocities(w)

    def move_right(self, v, theta):
        p_dot = [v, 0.0, 0.0]
        w = self.inverse_kinematics(p_dot, theta)
        self.set_angular_velocities(w)

    def rotate_CCW(self, w, theta):
        p_dot = [0.0, 0.0, w] #<- positive since when testing on bot, with w set to negative the robot would rotate CW instead of CCW
        w = self.inverse_kinematics(p_dot, theta)
        self.set_angular_velocities(w)

    def rotate_CW(self, w, theta):
        p_dot = [0.0, 0.0, -w] #<- negative since when testing on bot, with w set to positive the robot would rotate CCW instead of CW
        w = self.inverse_kinematics(p_dot, theta)
        self.set_angular_velocities(w)

    def HMatrix(self,q):
        H = array([[cos(q[2]), -sin(q[2]), q[0]], [sin(q[2]), cos(q[2]), q[1]], [0, 0, 1]])
        return H

    def Vraw_to_distance(self,Vraw):
        d = 0.6209*exp(-0.05396*sqrt(Vraw))
        return d

In [ ]:
#lab4demo.py (WORKING)

from time import time
from math import sqrt, pi, sin, cos, atan2
from numpy import array, dot, zeros
from numpy.linalg import norm

from myRobot import *
import sys
# Set up sampling period T_s and stopping time T_f
T_s = 0.02
T_f = 30

# Set up initial pose
x_0 = 0
y_0 = 0
theta_0 = 0

# Set up goal position
x_f = 0.7
y_f = 1.0

# Set up p_0 and p_f
p_0 = array([x_0, y_0, theta_0])
p_f = array([x_f, y_f, pi/2])

# Set up error tolerance
epsilon = 0.005

# set up d_free and d_min
d_min = 0.08
d_free = 1.25*d_min

# Set up controller gains
k_rho = 0.5
k_beta = -0.5
k_alpha = (2.0/pi)*k_rho - (5.0/3.0)*k_beta + 0.5

# Initialize vector pr_dot to be used for inverse kinematics
pr_dot = zeros(shape=(3,1))

# Initialize vector d for storing the sensor measurements d_i
d = array([10.0, 10.0, 10.0, 10.0, 10.0, 10.0]).T

# Initialize u_iR
u_iR = zeros(shape=(3,6))

# Initialize sensor frame location
R = 0.13
halfR = R/2.0
root3Rover2 = sqrt(3.0)*R/2.0

# Sensor frame location as per Table 4.1
sensor_loc = array([
   [-halfR, -root3Rover2, -(2.0/3.0)*pi],
   [-root3Rover2, halfR, (5/6)*pi],
   [0, R, 1/2*pi],
   [halfR, root3Rover2, 1/3*pi],
   [root3Rover2, halfR, 1/6*pi],
   [root3Rover2, -halfR, -1/6*pi]
])

# open a file to store robot pose for plotting later
f = open('pose.csv', 'w')

# Initial robot and other variables
robot = myRobot(T_s)

robot.initialize(theta_0)

H_R_Si_list = []
for i in range(0, 6):
    q = sensor_loc[i]  # Ensure q is in the format HMatrix expects
    try:
        H_R_Si = robot.HMatrix(q)  # Call the method with the correct arguments
        H_R_Si_list.append(H_R_Si)
    except TypeError as e:
        print("Error processing sensor_loc[{}]: {}".format(i,e))

H_R_Si_array = array(H_R_Si_list)

p = p_0
dp = p_f - p_0
rho = norm(dp[0:2])
goal_reached = False
elapsed_time = 0.0

# Set start time
start_time = time()


# Control loop
while (not goal_reached) and (elapsed_time < T_f):

    robot.get_readings_update()

    # save data to file for plotting
    f.write('%10.3e %10.3e % 10.3e %10.3e %10.3e\n' % (elapsed_time, p[0], p[1], p[2], rho))

    x = p[0]
    y = p[1]
    theta = p[2]

    # Use forward kinematics to get p_dot
    p_dot = robot.forward_kinematics(robot.angular_velocities, theta)

    # Determine angle alpha (remember to use atan2 instead of atan)
    alpha = atan2(y_f - y, x_f - x) - theta

    d_si_list = []
    d_accepted = []
    u_R_list = []
    d_R_list = []
    accepted_indices = []

    collision_exist = False

    # Get sensor measurements w.r.t. sensor frames
    for i in range(6):
        d_measured = robot.Vraw_to_distance(robot.ir_sensors_raw_values[i])
        d[i] = d_measured

        if d_measured <= d_min:
            collision_exist = True

    if collision_exist:
        for i in range(6):
            if d_measured > d_free:
                d_T = sum(d_accepted)
                for i in range(6):

                    # Find collision-free paths
                    if d_measured > d_free:
                        d_accepted.append(d_measured)
                        accepted_indices.append(i)

                        # Convert this path to vector d_Si
                        d_si = array([d_measured, 0, 1]).T
                        var = robot.HMatrix(sensor_loc[i])

                        # Transform from Si to R frame
                        d_R_i = dot(var, d_si)
                        d_R_list.append(d_R_i)

        # Calculate unit path vectors
        u_R_list = [d_R_i / norm(d_R_list) for d_R_i in d_R_list]

        # Calculate weighted sum of path vectors
        u_R = sum([d[i] / d_T * u_R_list[i] for i in accepted_indices])

        # Transform R to base link frame
        H_0_R = robot.HMatrix(p)
        u_0 = dot(H_0_R, u_R)

        # Set temporary goal
        p_T = d_free * u_0

        alpha_bar = atan2(p_T[1] - y, p_T[0] - x) - theta
        alpha = alpha_bar

        rho_bar = norm([p_T[1] - y, p_T[0] - x])
        rho = rho_bar

    beta = -(theta + alpha)

    # Determine linear and angular velocities of the robot body (v, w)
    # using the control law given in Eqs. (4.12) and (4.13)
    v = k_rho * rho
    w = k_alpha * alpha + k_beta * beta

    # Determine pr_dot (temporary velocity desired)
    pr_dot[0] = v * cos(theta)
    pr_dot[1] = v * sin(theta)
    pr_dot[2] = w

    # Now use Inverse Kinematics to determine wheel reference velocities
    wheel_ref_vel = robot.inverse_kinematics(pr_dot, theta)

    # Apply motor limits
    wheel_ref_vel = robot.motor_limit(wheel_ref_vel, 5.0 * pi)

    # Execute motion
    robot.set_angular_velocities(wheel_ref_vel)

    # Odometry update
    p = p + p_dot * T_s

    # Replace calculated update for theta with measured value from IMU
    p[2] = robot.orientation

    # Check to see if goal is reached
    dp = p_f - p
    rho = norm(dp[0:2])
    goal_reached = (rho <= epsilon)

    # Time update
    elapsed_time = time() - start_time


print('ELAPSED TIME = %s rho = %s' % (elapsed_time, rho))

# Either goal is reached or current_time > T_f
if goal_reached:
   print('Goal is reached, error norm is', rho)
else:
   print('Failed to reach goal, error norm is', rho)

robot.stop()
robot.close()

f.close()